In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import yfinance as yf
import keras 
import os
import tensorflow
import datetime 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import random
import scipy.stats as st

In [18]:
def DataPrep(ticker,daysago,split_pct):
    #simple data prep
    df = yf.Ticker(ticker.upper()).history(period='max')
    df = df[['Close']]
    # df.plot(figsize=(10,10))
    for i in range(daysago, -1,-1):
        df[f'{i} daysago'] = df['Close'].shift(i)
        df = df.copy()
    df = df.dropna()
    df = df.drop(columns=['Close'])
    #ok so if i have tmrs price be the mean of the prob distro,
    # using stddev i can map a psuedo confidence interval type shit

    #if i use a normal distro, unlike the digits recognition ml, 
    #whos output is discrete(a vector of probabilities),
    #my output would have to resemble a continuous function, 
    #to simulate the normal distro

    e = pd.DataFrame()
    e['stddev'] = np.std(df,axis=1)
    e['mean'] = np.mean(df,axis=1)
    e['z'] = (df['0 daysago'].shift(-1) - e['mean'])/e['stddev']
    df = df.dropna()
    e = e.dropna()
    Y = []
    for yah in e['z']:
        if yah < -3:
            Y.append([1,0,0,0,0,0,0,0])
        elif yah >= -3 and yah <-2:
            Y.append([0,1,0,0,0,0,0,0])
        elif yah >= -2 and yah <-1:
            Y.append([0,0,1,0,0,0,0,0])
        elif yah >= -1 and yah <0:
            Y.append([0,0,0,1,0,0,0,0])
        elif yah >= 0 and yah <1:
            Y.append([0,0,0,0,1,0,0,0])    
        elif yah >= 1 and yah <2:
            Y.append([0,0,0,0,0,1,0,0])
        elif yah >= 2 and yah <3:
            Y.append([0,0,0,0,0,0,1,0])
        elif yah >= 3:
            Y.append([0,0,0,0,0,0,0,1])
    Y = np.array(Y)
    
    # df = df.drop(columns=['stddev'])
    # df = df.drop(columns=['mean'])
    # df = df.drop(columns=['z'])
    sc = MinMaxScaler(feature_range=(0,1))
    sc2 = MinMaxScaler(feature_range=(0,1))
    scaled = sc.fit_transform(df)
    X= scaled[:,:daysago+1]
    split = int(len(X)*split_pct)
    split2 = int(len(Y)*split_pct)
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split2], Y[split2:]
    X_train, X_test, Y_train, Y_test = X_train.reshape((-1,daysago+1,1)), X_test.reshape((-1,daysago+1,1)), Y_train.reshape((-1,8,1)), Y_test.reshape((-1,8,1))
    return X_train, X_test, Y_train, Y_test


In [4]:
# Output should be a vector, 13 Xs maybe, like [-3, -2.5, -2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5, 3]
# being the stddevs from tmrs price, and the values would be the %prob
# [0.ugly, 0.decent, 0.highest, 0.decent, 0.ugly] to simplify
# function to convert might look smth like this:
DataPrep('aapl',100,0.8)
# output = [0.01,0.1,0.78,0.1,0.01]

# #for list
# max_index = np.argmax(output)

# z = st.norm.ppf(max(output))



# df = yf.Ticker('AAPL').history(start='2010-01-01',end='2010-01-20')
# df = np.array(df['Close'])
# sigma = np.std(df)
# mu = np.mean(df)
# sigma, mu,df

# z = (df[-1]-mu)/sigma
# st.norm.cdf(z)

# np.floor(z), np.ceil(z)


#wait a minute price data isnt normally distributed, its lognormal?
#i think i need to log transform the data to make it normally distributed
#price %return is normally distributed(i think?), so i can use that to predict price, but ml is shit with it so idk


(array([[[0.00030958],
         [0.00028342],
         [0.00024853],
         ...,
         [0.00025131],
         [0.00024031],
         [0.00024581]],
 
        [[0.00028342],
         [0.00024853],
         [0.00025943],
         ...,
         [0.00024031],
         [0.00024581],
         [0.00024948]],
 
        [[0.00024853],
         [0.00025943],
         [0.00027252],
         ...,
         [0.00024581],
         [0.00024948],
         [0.00024031]],
 
        ...,
 
        [[0.14400742],
         [0.14382523],
         [0.14332406],
         ...,
         [0.11324104],
         [0.11468164],
         [0.11518439]],
 
        [[0.14382523],
         [0.14332406],
         [0.14297095],
         ...,
         [0.11468164],
         [0.11518439],
         [0.1136858 ]],
 
        [[0.14332406],
         [0.14297095],
         [0.13941723],
         ...,
         [0.11518439],
         [0.1136858 ],
         [0.11477834]]]),
 array([[[0.14297095],
         [0.13941723],
         

In [14]:

X_train = []
X_test = []
Y_train = []
Y_test = []
X_train,X_test,Y_train,Y_test = np.array(X_train), np.array(X_test), np.array(Y_train), np.array(Y_test)
X_train,X_test,Y_train,Y_test = X_train.reshape(-1,101,1),X_test.reshape(-1,101,1),Y_train.reshape(-1,8,1),Y_test.reshape(-1,8,1)


sp500list = pd.read_csv("4th attempt total/SP500.csv")
ListOfTickers = []
for i in sp500list['Symbol']:
    ListOfTickers.append(i)

ListOfTickers.remove('BRK.B')
ListOfTickers.remove('BF.B')
ListOfTickers.remove('GEV')
ListOfTickers.remove('SOLV')
ListOfTickers.remove('HUBB')
ListOfTickers.remove('BRO')
# ListOfTickers = ['aapl']

for ticker in ListOfTickers:
    print(ticker)
    dataprep = DataPrep(ticker,100,0.9)
    X_train = np.append(X_train,dataprep[0],axis=0)
    X_test = np.append(X_test,dataprep[1],axis=0)
    Y_train = np.append(Y_train,dataprep[2],axis=0)
    Y_test = np.append(Y_test,dataprep[3],axis=0)

X_train.shape,X_test.shape,Y_train.shape,Y_test.shape


MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BLDR
BG
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CHRW
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUM
HBAN
HII
IBM
IEX

((3772650, 101, 1), (419419, 101, 1), (3772650, 8, 1), (418920, 8, 1))

In [6]:
model3 = Sequential()
model3.add(InputLayer((101, 1)))
model3.add(LSTM(66))
model3.add(Dense(33, 'sigmoid'))
model3.add(Dense(8, 'softmax'))

cp1 = ModelCheckpoint('model/6th.keras', save_best_only=False)
model3.compile(loss='mse',optimizer=Adam(learning_rate=0.001), metrics=['mse'])
model3.fit(X_train, Y_train, epochs=2, batch_size=256, callbacks=[cp1])


Epoch 1/2
16358/16358 ━━━━━━━━━━━━━━━━━━━━ 1600s 98ms/step - loss: 0.0974 - mse: 0.0974
Epoch 2/2
16358/16358 ━━━━━━━━━━━━━━━━━━━━ 1593s 97ms/step - loss: 0.0687 - mse: 0.0687


13107/13107 ━━━━━━━━━━━━━━━━━━━━ 117s 9ms/step
(418920, 1) (419419,)


In [15]:
model3 = load_model('model/6th.keras')
y_pred = model3.predict(X_test)
y_pred_acc = np.argmax(y_pred,axis=1)
Y_test_acc = np.argmax(Y_test,axis=1)
print(Y_test_acc.shape,y_pred_acc.shape)

# avg_error = (np.sum(np.sqrt((Y_test - y_pred)**2)))/len(Y_test)*100
# print(avg_error)
# avg_error = (np.sum(y_pred - Y_test))/len(Y_test)*100
# print(avg_error)
# std,mean,z

sp500_y_pred,sp500_Y_test = pd.DataFrame(y_pred_acc),pd.DataFrame(Y_test_acc)
sp500_y_pred.plot(figsize=(16,8))
for i in range(0,sp500_y_pred.shape[1]): 
    oneday= pd.concat([sp500_y_pred[i],sp500_Y_test[i]],axis=1)
    oneday.plot(figsize=(16,8),color = ('green','red'))

    

In [17]:
#this is a test for accuracy, not mse or mae or rmse
fawk = y_pred[:-499] == Y_test.reshape(-1,8)
fawk = fawk.astype(int)
fawk = np.sum(fawk)/len(fawk)*100
fawk

0.0

In [75]:
# this is a mae test
avg_error = (np.sum(np.sqrt((sp500_Y_test - sp500_y_pred)**2)))/len(Y_test)*100
avg_error

/Users/milagrochen/Documents/GitHub/Stock-Predict-Summer-24--Attempts/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0    1.152357
dtype: float64

<Axes: >

In [66]:

sp500_y_pred['acc'] = sp500_y_pred[2]*sp500_y_pred[0] + sp500_y_pred[1]
sp500_Y_test['acc'] = sp500_Y_test[2]*sp500_Y_test[0] + sp500_Y_test[1]
sp500_y_pred,sp500_Y_test
acc = pd.concat([sp500_y_pred['acc'],sp500_Y_test['acc']],axis=1)
acc.plot(figsize=(16,8),color = ('green','red'))